# NMCT-Box Jupyter demo

- Via de Jupyter webinterface kan je live code uitvoeren op de NMCT-box. 
- Het resultaat kan je zo opslaan als "notebook", maar ook als zuiver Python-bestand
- Of exporteren als HTML, Markdown, PDF,... 

## NMCT Box basics

In [1]:
import nmct.box

acc = nmct.box.get_accelerometer()
print(acc.measure())
print(acc.tilt())


Acceleration(x=10.5127, y=1.6475, z=-0.1569)
Tilt(roll=8.90570078142508, pitch=81.05367423241354)


In [2]:
devices = nmct.box.list_onewire_ids()
print(devices)
if len(devices) == 1:
    t = nmct.box.get_thermometer(devices[0]).measure()
    print("It's {} \N{DEGREE SIGN}C".format(t))

['28-0417b29bccff']
It's 20.312 °C


In [3]:
import aiy.voicehat
from datetime import datetime
button = aiy.voicehat.get_button()
print("It's now {}".format(datetime.utcnow()))
button.wait_for_press()
print("You pressed at {}!".format(datetime.utcnow()))


It's now 2018-04-19 13:36:14.034558
You pressed at 2018-04-19 13:36:23.462888!


## Watson Conversation

In [1]:
from nmct import watson

# workspace_id = "9de1dde4-e3a6-406b-9913-82e5ce74f64c"
# cnv = watson.get_conversation(workspace_id)

cnv = watson.get_conversation()
intents, entities, output = cnv.message("Hey babe!")
print(output["text"][0])


Hi. It looks like a nice drive today. What would you like me to do?  


In [ ]:
while not cnv.finished:
    print("\n------------------------------------------------------------------------------------")
    text = input()
    intents, entities, output = cnv.message(text)
    print("Intents: {} ".format(intents))
    print("Entities: {} ".format(entities))
    print("Output: {} ".format(output))
    print
    if "goodbyes" in intents:
        cnv.finish()



------------------------------------------------------------------------------------
turn on the lights
Intents: ['turn_on'] 
Entities: [{'entity': 'appliance', 'value': 'lights', 'location': [12, 18], 'confidence': 1}] 
Output: {'nodes_visited': ['Entry Point For On Off Commands', 'node_2_1467232480480', 'Appliance On Off Check'], 'log_messages': [], 'action': {'lights_on': ''}, 'text': ["I'll turn on the lights for you."]} 

------------------------------------------------------------------------------------


## Speech recognition

In [ ]:
from nmct import box, watson, Color, Palette
import aiy.voicehat

stt = watson.get_recognizer()
button = aiy.voicehat.get_button()
ring = box.get_pixel_ring()
running = True


def stop():
    running = False


button.on_press(stop)

while running:
    word = stt.recognize()
    
    if word in Palette:
        color = Color.by_name(word)
        ring.queue_effect("fill", color)
    else:
        nmct.box.play_sound("glass")
